In [6]:
from dotenv import load_dotenv
load_dotenv()



True

In [9]:
import os 
os.environ["HF_TOKEN"] =os.getenv("HF_TOKEN")

In [15]:
from langchain_huggingface import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

/Users/ebythomas/agentic_ai_2.0/venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [16]:
embeddings.embed_query("Hello, world!")

[-0.038177140057086945,
 0.032911088317632675,
 -0.005459381267428398,
 0.01436994131654501,
 -0.04029103368520737,
 -0.11653246730566025,
 0.0316876657307148,
 0.0019117513438686728,
 -0.04262230917811394,
 0.029168114066123962,
 0.042426690459251404,
 0.032041676342487335,
 0.02984473668038845,
 0.010980335995554924,
 -0.053939640522003174,
 -0.05027727782726288,
 -0.02350783906877041,
 0.010779364965856075,
 -0.1377079337835312,
 0.004115015733987093,
 0.02933305688202381,
 0.06684114784002304,
 -0.015389405190944672,
 0.04843764379620552,
 -0.0881497710943222,
 -0.01272682473063469,
 0.04140901938080788,
 0.040831517428159714,
 -0.0501558855175972,
 -0.058125052601099014,
 0.048801489174366,
 0.06889013946056366,
 0.05874691531062126,
 0.008731027133762836,
 -0.01591823622584343,
 0.08514197915792465,
 -0.07814742624759674,
 -0.07751677185297012,
 0.020723801106214523,
 0.016194235533475876,
 0.03251060098409653,
 -0.053488872945308685,
 -0.062228795140981674,
 -0.02431465312838554

In [17]:
from sklearn.metrics.pairwise import cosine_similarity

In [19]:
documents=["what is the capital of USA?",
           "Who is the presidant of USA",
           "Who is the prime minister of India"]
           

In [20]:
my_query="Donald Trump is the presidant of USA"


In [21]:

document_embedding = embeddings.embed_documents(documents)

In [22]:
document_embedding

[[0.13646377623081207,
  -0.005729749333113432,
  -0.025343354791402817,
  0.07661077380180359,
  -0.05113113671541214,
  -0.03266711160540581,
  0.01869008131325245,
  -0.03566405549645424,
  -0.02634115144610405,
  -0.02951105497777462,
  -0.02450016513466835,
  -0.06638791412115097,
  -0.004297670442610979,
  -0.007684133946895599,
  -0.05228116735816002,
  -0.05827875807881355,
  0.02489960566163063,
  -0.02401776798069477,
  0.05123415216803551,
  0.007984810508787632,
  0.024020347744226456,
  -0.02073512226343155,
  0.012419339269399643,
  -0.026782307773828506,
  0.06189240887761116,
  0.007056963164359331,
  0.0015273187309503555,
  -0.003130122786387801,
  0.0038281830493360758,
  -0.03322750702500343,
  0.01280911173671484,
  -0.10833609849214554,
  0.06761717051267624,
  -0.02913142926990986,
  -0.01266469992697239,
  -0.009346024133265018,
  0.06864696741104126,
  0.005309020169079304,
  0.07964584231376648,
  0.01357949897646904,
  0.019175559282302856,
  -0.0003756472724

In [23]:
query_embedding = embeddings.embed_query(my_query)

In [24]:
len(query_embedding)

384

In [27]:
cosine_similarity([query_embedding],document_embedding)


array([[0.3575966 , 0.82672138, 0.21090333]])

In [28]:
from sklearn.metrics.pairwise import euclidean_distances
euclidean_distances([query_embedding],document_embedding)

array([[1.1334932 , 0.58869111, 1.25626168]])

| Metric            | Similarity Score Range | Behavior                              |
| ----------------- | ---------------------- | ------------------------------------- |
| Cosine Similarity | \[-1, 1]               | Focuses on angle only |
| L2 Distance       | \[0, ∞)                | Focuses on **magnitude + direction**  |


In [29]:
import faiss
from langchain_community.vectorstores import FAISS
from langchain_community.docstore.in_memory import InMemoryDocstore


In [31]:
index = faiss.IndexFlatL2(384)
index


<faiss.swigfaiss.IndexFlatL2; proxy of <Swig Object of type 'faiss::IndexFlatL2 *' at 0x151274330> >

In [32]:
vector_store = FAISS(
    embedding_function= embeddings
    ,index=index,
    docstore=InMemoryDocstore(),
    index_to_docstore_id={},
)

In [33]:
vector_store.add_texts(["AI is future","AI is powerfull","Dogs are cute"])

['849c6bd8-586f-4341-b1f9-0d1aec792115',
 'd42d0077-b2ec-414c-9bb6-058f380969ae',
 '4861095e-4024-453b-a9dc-74a8241ada86']

In [34]:
vector_store.index_to_docstore_id

{0: '849c6bd8-586f-4341-b1f9-0d1aec792115',
 1: 'd42d0077-b2ec-414c-9bb6-058f380969ae',
 2: '4861095e-4024-453b-a9dc-74a8241ada86'}

In [35]:
results = vector_store.similarity_search("Tell me about AI", k=3)

In [36]:
results


[Document(id='d42d0077-b2ec-414c-9bb6-058f380969ae', metadata={}, page_content='AI is powerfull'),
 Document(id='849c6bd8-586f-4341-b1f9-0d1aec792115', metadata={}, page_content='AI is future'),
 Document(id='4861095e-4024-453b-a9dc-74a8241ada86', metadata={}, page_content='Dogs are cute')]

In [38]:
results = vector_store.similarity_search("Tell me about AI", k=1)
results

[Document(id='d42d0077-b2ec-414c-9bb6-058f380969ae', metadata={}, page_content='AI is powerfull')]

| Feature               | `Flat`                | `IVF` (Inverted File Index)        | `HNSW` (Graph-based Index)          |
| --------------------- | --------------------- | ---------------------------------- | ----------------------------------- |
| Type of Search     | Exact                 | Approximate (cluster-based)        | Approximate (graph-based traversal) |
| Speed               | Slow (linear scan)    | Fast (search only in top clusters) | Very Fast (graph walk)              |


| Dataset Size              | Recommended Index                 |
| ------------------------- | --------------------------------- |
| UPTO 1L                     | `IndexFlatL2` or `IndexFlatIP`    |
| UPTO 1M                  | `IndexIVFFlat` or `IndexHNSWFlat` |
| > 1M                      | `IndexIVFPQ` or `IndexHNSWFlat`   |


In [39]:
# from uuid import uuid4
from langchain_core.documents import Document

document_1 = Document(
    page_content="I had chocolate chip pancakes and scrambled eggs for breakfast this morning.",
    metadata={"source": "tweet"},
)

document_2 = Document(
    page_content="The weather forecast for tomorrow is cloudy and overcast, with a high of 62 degrees.",
    metadata={"source": "news"},
)

document_3 = Document(
    page_content="Building an exciting new project with LangChain - come check it out!",
    metadata={"source": "tweet"},
)

document_4 = Document(
    page_content="Robbers broke into the city bank and stole $1 million in cash.",
    metadata={"source": "news"},
)

document_5 = Document(
    page_content="Wow! That was an amazing movie. I can't wait to see it again.",
    metadata={"source": "tweet"},
)

document_6 = Document(
    page_content="Is the new iPhone worth the price? Read this review to find out.",
    metadata={"source": "website"},
)

document_7 = Document(
    page_content="The top 10 soccer players in the world right now.",
    metadata={"source": "website"},
)

document_8 = Document(
    page_content="LangGraph is the best framework for building stateful, agentic applications!",
    metadata={"source": "tweet"},
)

document_9 = Document(
    page_content="The stock market is down 500 points today due to fears of a recession.",
    metadata={"source": "news"},
)

document_10 = Document(
    page_content="I have a bad feeling I am going to get deleted :(",
    metadata={"source": "tweet"},
)

documents = [
    document_1,
    document_2,
    document_3,
    document_4,
    document_5,
    document_6,
    document_7,
    document_8,
    document_9,
    document_10,
]

In [41]:
index = faiss.IndexFlatIP(384)
vector_store = FAISS(
    embedding_function= embeddings,
    index=index,
    docstore= InMemoryDocstore(),
    index_to_docstore_id={}
)

In [43]:
vector_store.add_documents(documents=documents)


['8f2ee400-f03b-42e7-a791-394c9708bc5a',
 'a86f6860-9122-4a84-be16-c05bf7192fd0',
 '49be4691-4f9d-41e3-ac1d-e7aaf80fad78',
 '36ec1656-bca1-47cd-9649-23749d69730e',
 '02b20dae-8331-42a2-be4f-60b5b0623b99',
 '72c7581f-30c0-4733-b1ab-db8a25902daa',
 'eb6796d9-339d-417e-ad6c-32516161104b',
 '9c7f94d9-bb93-433e-bbe5-74bc9af29855',
 '07398008-3cdd-4286-aaa1-9c4fe24f9684',
 'e7ea8478-315b-4762-9a69-b9f700d5cdc0']

In [44]:
vector_store.similarity_search(
    "langchain  provides abstractions to make wokring with llms easy",
    k=2
)

[Document(id='49be4691-4f9d-41e3-ac1d-e7aaf80fad78', metadata={'source': 'tweet'}, page_content='Building an exciting new project with LangChain - come check it out!'),
 Document(id='9c7f94d9-bb93-433e-bbe5-74bc9af29855', metadata={'source': 'tweet'}, page_content='LangGraph is the best framework for building stateful, agentic applications!')]

In [46]:
vector_store.similarity_search(
     "langchain  provides abstractions to make wokring with llms easy",
     filter={"source":"news"}
)

[Document(id='36ec1656-bca1-47cd-9649-23749d69730e', metadata={'source': 'news'}, page_content='Robbers broke into the city bank and stole $1 million in cash.'),
 Document(id='a86f6860-9122-4a84-be16-c05bf7192fd0', metadata={'source': 'news'}, page_content='The weather forecast for tomorrow is cloudy and overcast, with a high of 62 degrees.'),
 Document(id='07398008-3cdd-4286-aaa1-9c4fe24f9684', metadata={'source': 'news'}, page_content='The stock market is down 500 points today due to fears of a recession.')]

In [49]:
 results =vector_store.similarity_search(
     "langchain  provides abstractions to make wokring with llms easy",
     filter={"source":{"$eq":"tweet"}}
        )

In [52]:
results[0].page_content

'Building an exciting new project with LangChain - come check it out!'

In [53]:
retriver = vector_store.as_retriever(search_kwargs={"k":3})

In [54]:
retriver.invoke("langchain  provides abstractions to make wokring with llms easy")

[Document(id='49be4691-4f9d-41e3-ac1d-e7aaf80fad78', metadata={'source': 'tweet'}, page_content='Building an exciting new project with LangChain - come check it out!'),
 Document(id='9c7f94d9-bb93-433e-bbe5-74bc9af29855', metadata={'source': 'tweet'}, page_content='LangGraph is the best framework for building stateful, agentic applications!'),
 Document(id='eb6796d9-339d-417e-ad6c-32516161104b', metadata={'source': 'website'}, page_content='The top 10 soccer players in the world right now.')]

In [ ]:
InMemory
ondisk
cloud


In [55]:
vector_store.save_local("today's class faiss index")

In [56]:
new_vector_store= FAISS.load_local(
    "today's class faiss index",embeddings,allow_dangerous_deserialization= True
)

In [58]:
new_vector_store.similarity_search("langchain")

[Document(id='49be4691-4f9d-41e3-ac1d-e7aaf80fad78', metadata={'source': 'tweet'}, page_content='Building an exciting new project with LangChain - come check it out!'),
 Document(id='9c7f94d9-bb93-433e-bbe5-74bc9af29855', metadata={'source': 'tweet'}, page_content='LangGraph is the best framework for building stateful, agentic applications!'),
 Document(id='02b20dae-8331-42a2-be4f-60b5b0623b99', metadata={'source': 'tweet'}, page_content="Wow! That was an amazing movie. I can't wait to see it again."),
 Document(id='eb6796d9-339d-417e-ad6c-32516161104b', metadata={'source': 'website'}, page_content='The top 10 soccer players in the world right now.')]

In [63]:
from langchain_community.document_loaders import PyPDFLoader

In [64]:
FILE_PATH = r"/Users/ebythomas/agentic_ai_2.0/2-Langchain Basics/data/llama2.pdf"


In [65]:
loader = PyPDFLoader(FILE_PATH)

In [66]:
len(loader.load())

77

In [67]:
pages = loader.load()

In [69]:
len(pages)

77

In [70]:
pages =[]
async for page in loader.alazy_load():
    pages.append(page)

In [71]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [73]:
splitter = RecursiveCharacterTextSplitter(
    chunk_size= 500,
    chunk_overlap=50
)

In [74]:
split_docs = splitter.split_documents(pages)

In [76]:
len(split_docs)

615

In [77]:
index = faiss.IndexFlatL2(384)
vector_store = FAISS(
    embedding_function=embeddings,
    index=index,
    docstore=InMemoryDocstore(),
    index_to_docstore_id={}
)

In [78]:
vector_store.add_documents(documents=split_docs)

['4fd81a2c-9cb8-49f4-854f-fde3589ba56f',
 '6ff18002-e2c1-4362-a168-f01f9f9a9845',
 '44beda2c-9d6f-4f2d-a145-3e6f48a37067',
 '744a8d07-9f32-4a47-8d1f-0848fe200992',
 '2c138ede-07d8-476a-a3cf-da6f8e635876',
 '700f2301-e53d-4968-b59f-d4d9606fe525',
 'b41c3595-6650-4e90-ad7c-5a7200c7bc66',
 'e9c2a66f-425c-429a-8e38-14def1a75d8a',
 'bf55f1f0-593b-486b-84b1-8fb17636ac72',
 'e5fe3af6-9814-4401-bdad-fa8e5725e048',
 'af2b1dfa-d3bb-4959-a129-19e7a674dbec',
 '9ba8f7ca-43a2-4e53-8d9e-202155b05031',
 '3f5c26bb-efec-43e2-8387-6030afd0258c',
 'd8610ed0-209e-4708-99e7-9562523da833',
 'bd0f1b5a-787e-4d76-932f-3f3304669c4c',
 '0976edea-0475-4feb-9906-2e6fe5d625c7',
 '33cbd7be-8adb-4d98-bcb0-e1aa923ecb39',
 'c5e4d700-252d-4d88-b4a6-7fa93534e8d1',
 'd2ede01f-0ee2-4d6e-a614-ccd96fefce78',
 'c180d52a-bd98-4ce4-b6e6-c7c3487d11da',
 '5864eae7-b68f-4d4a-a5f8-e90aefe45caa',
 '6a44c96a-9539-4e0e-9182-4eee85cb7487',
 '89b3da01-a5e7-4db6-953e-421f39501713',
 'c93e8168-df23-45a7-923b-059e4b49d059',
 '3e2a96a0-af71-

In [79]:
retreiver = vector_store.as_retriever(
    search_kwargs={"k":10}
)

In [80]:
retreiver.invoke("what is llama")

[Document(id='3e2a96a0-af71-474e-a89e-3282561c9487', metadata={'producer': 'pdfTeX-1.40.25', 'creator': 'LaTeX with hyperref', 'creationdate': '2023-07-20T00:30:36+00:00', 'author': '', 'keywords': '', 'moddate': '2023-07-20T00:30:36+00:00', 'ptex.fullbanner': 'This is pdfTeX, Version 3.141592653-2.6-1.40.25 (TeX Live 2023) kpathsea version 6.3.5', 'subject': '', 'title': '', 'trapped': '/False', 'source': '/Users/ebythomas/agentic_ai_2.0/2-Langchain Basics/data/llama2.pdf', 'total_pages': 77, 'page': 3, 'page_label': '4'}, page_content='work (Section 6), and conclusions (Section 7).\n‡https://ai.meta.com/resources/models-and-libraries/llama/\n§We are delaying the release of the 34B model due to a lack of time to sufficiently red team.\n¶https://ai.meta.com/llama\n‖https://github.com/facebookresearch/llama\n4'),
 Document(id='3faccea0-3bd6-4a95-9982-98f092d08e5c', metadata={'producer': 'pdfTeX-1.40.25', 'creator': 'LaTeX with hyperref', 'creationdate': '2023-07-20T00:30:36+00:00', 'aut

In [81]:
from langchain_groq import ChatGroq

model = ChatGroq(model="gemma2-9b-it")

In [82]:
from langchain import hub 
prompt = hub.pull("rlm/rag-prompt")

In [83]:
import pprint 

In [84]:
pprint.pprint(prompt.messages)

[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"), additional_kwargs={})]


[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"), additional_kwargs={})]

In [85]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

In [ ]:
context(retriever),prompt(hub),model(google),parser(langchain)

In [89]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

In [90]:
rag_chain = (
    {"context":retreiver | format_docs,"question": RunnablePassthrough()}
    |prompt
    |model
    | StrOutputParser()
)

In [91]:
rag_chain.invoke("what is llama model")

'Llama 2 is a large language model (LLM) released by Meta.  It is available in both pretrained and tuned versions, with the tuned versions being specifically designed for chat applications. Llama 2 is intended for both commercial and research use in English. \n\n\n'